#### Dataset Replication  
Use predefined SQL table copies to partion and cluster new tables

In [1]:
import bq, bqa, config, sfa
from google.cloud import bigquery

In [2]:
from importlib import reload

#### Set Source and Destination

In [3]:
source_dataset      = "ds_1GB_01"
destination_dataset = "ds_1GB_02"

#### Snowflake Database Copy

In [4]:
source_dataset.upper()

'DS_1GB_01'

In [5]:
%%time
sf = sfa.SFTPC(test="ds",  # class attributes are placeholders
               scale=1,
               cid="0X",
               warehouse="TEST9000", 
               desc="dev_clone_test",
               verbose=True,
               verbose_query=False)
sf.database = source_dataset.upper()
sf.connect()

query_result = sf.copy(destination_dataset.upper())
print(query_result.sfqid)

sf.close()

Snowflake configuration
Username: colin
Account:  wja13212

01957529-029d-3fa2-0009-c0830006de8a
CPU times: user 213 ms, sys: 8.4 ms, total: 221 ms
Wall time: 13.9 s


In [6]:
query_result

In [7]:
query_result.sfqid

'01957529-029d-3fa2-0009-c0830006de8a'

#### BigQuery Dataset Copy

In [8]:
# schema definition copies AND applies clustering/partitioning
schema_name = "ds_01A.sql"

In [9]:
query_text = bqa.create_table_remix(schema_name=schema_name,
                                    source=source_dataset,
                                    destination=destination_dataset,
                                    dot=True)

In [10]:
[print(line) for line in query_text.split("\n")[0:10]];

-- DS test version 02
-- cluster on time
-- 
create table `ds_1GB_02.dbgen_version`
as
select * from `ds_1GB_01.dbgen_version`;

create table `ds_1GB_02.customer_address`
as
select * from `ds_1GB_01.customer_address`;


In [11]:
bq.create_dataset(destination_dataset)

Dataset(DatasetReference('tpc-benchmarking-9432', 'ds_1GB_02'))

In [12]:
%%time
query_job = bq.query(query_text=query_text,
                     project=config.gcp_project.lower(),
                     dataset=destination_dataset,
                     dry_run=False,
                     use_cache=False)
query_job.result()

CPU times: user 208 ms, sys: 1.2 ms, total: 209 ms
Wall time: 3min 57s
